## CIFAR 10 Dataset

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

True

In [4]:
# Parameters
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                   (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./', download=True, train=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./', download=True, train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [6]:
classes = {'plane', 'car', 'bird', 'cat', 'deer',
          'dog', 'frog', 'horse', 'ship', 'truck'}

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)        
        
    def forward(self, X):
        out = self.conv1(X)
        out = self.pool(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.pool(out)
        out = self.relu(out)
        
        out = out.reshape(-1, 16 * 5 * 5)
        
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)      
        
        return out

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_iters = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        y_pred = model(images)
        
        loss = criterion(y_pred, labels)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if (i + 1) % 2000 == 0:
            print(f"Epoch: {epoch + 1}/{num_epochs}, Step: {i + 1}/{n_iters}, Loss: {loss.item()}")

with torch.no_grad():
    num_samples = 0
    num_correct = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)
        
        _, predictions = torch.max(output, dim=1)
        
        num_correct += (predictions == labels).sum().item()
        num_samples += labels.shape[0]
        
    accuracy = (num_correct / num_samples) * 100.0
    print(f"Accuracy: {accuracy}")

Epoch: 1/5, Step: 2000/12500, Loss: 2.302532196044922
Epoch: 1/5, Step: 4000/12500, Loss: 2.2630722522735596
Epoch: 1/5, Step: 6000/12500, Loss: 2.077075958251953
Epoch: 1/5, Step: 8000/12500, Loss: 2.140511989593506
Epoch: 1/5, Step: 10000/12500, Loss: 1.6865479946136475
Epoch: 1/5, Step: 12000/12500, Loss: 2.013993501663208
Epoch: 2/5, Step: 2000/12500, Loss: 1.6799960136413574
Epoch: 2/5, Step: 4000/12500, Loss: 1.576537847518921
Epoch: 2/5, Step: 6000/12500, Loss: 2.044679641723633
Epoch: 2/5, Step: 8000/12500, Loss: 2.0757956504821777
Epoch: 2/5, Step: 10000/12500, Loss: 1.3058042526245117
Epoch: 2/5, Step: 12000/12500, Loss: 1.3837724924087524
Epoch: 3/5, Step: 2000/12500, Loss: 1.1932172775268555
Epoch: 3/5, Step: 4000/12500, Loss: 1.535935640335083
Epoch: 3/5, Step: 6000/12500, Loss: 0.9015352725982666
Epoch: 3/5, Step: 8000/12500, Loss: 2.1275696754455566
Epoch: 3/5, Step: 10000/12500, Loss: 1.1868253946304321
Epoch: 3/5, Step: 12000/12500, Loss: 1.079467535018921
Epoch: 4/5, 